# Capstone Project 1 Machine Learning/Forecasting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA
from pandas.tools.plotting import autocorrelation_plot
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

In [2]:
# Read in dataset
df = pd.read_csv('../data/by_zip.csv', index_col = [0,1], header=[0], parse_dates=True)
df = df.loc[:,'2010-01-01':]
df.head()

2010-01-01  2010-02-01  2010-03-01  2010-04-01  \
CountyName RegionName                                                   
Alameda    94501         586200.0    585700.0    585400.0    585600.0   
           94502         550400.0    551600.0    550600.0    548800.0   
           94536         532300.0    535000.0    535700.0    535800.0   
           94538         444000.0    445100.0    443400.0    443300.0   
           94539         888600.0    891600.0    891000.0    893900.0   

                       2010-05-01  2010-06-01  2010-07-01  2010-08-01  \
CountyName RegionName                                                   
Alameda    94501         587200.0    590000.0    589000.0    586300.0   
           94502         550600.0    551900.0    548400.0    544300.0   
           94536         540500.0    544300.0    542500.0    538400.0   
           94538         447900.0    452300.0    452100.0    450800.0   
           94539         902700.0    907700.0    905700.0    902300.0   

                       2010-09-01  2010-10-01     ...      2017-08-01  \
CountyName RegionName                             ...                   
Alameda    94501         581000.0    576900.0     ...          970400   
           94502         543900.0    539800.0     ...          876300   
           94536         533000.0    528800.0     ...          963200   
           94538         449100.0    447900.0     ...          880900   
           94539         895700.0    889200.0     ...         1592800   

                       2017-09-01  2017-10-01  2017-11-01  2017-12-01  \
CountyName RegionName                                                   
Alameda    94501           982900      994200     1003400     1012700   
           94502           885000      897300      903700      906500   
           94536           973000      984100      996400     1010700   
           94538           890800      904100      919500      935800   
           94539          1604800     1621900     1640700     1659300   

                       2018-01-01  2018-02-01  2018-03-01  2018-04-01  \
CountyName RegionName                                                   
Alameda    94501          1027300     1042800     1052400     1054800   
           94502           913800      924200      923600      921000   
           94536          1030900     1052700     1066600     1079100   
           94538           955800      976100      989000      998900   
           94539          1685100     1713300     1729500     1736000   

                       2018-05-01  
CountyName RegionName              
Alameda    94501          1054100  
           94502           921400  
           94536          1092200  
           94538          1008300  
           94539          1742000  

[5 rows x 101 columns]

In [3]:
# Group by CountyName
df_grouped = df.groupby('CountyName').mean()
df_grouped.head()

,2010-01-01,2010-02-01,2010-03-01,2010-04-01,2010-05-01,2010-06-01,2010-07-01,2010-08-01,2010-09-01,2010-10-01,...,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01
CountyName,,,,,,,,,,,,,,,,,,,,,
Alameda,5.485732e+05,5.500756e+05,5.494244e+05,5.484707e+05,5.508220e+05,5.525634e+05,5.446524e+05,5.409714e+05,5.362214e+05,5.317405e+05,...,9.389721e+05,9.455116e+05,9.556558e+05,9.666628e+05,9.785814e+05,9.949140e+05,1.011658e+06,1.020351e+06,1.023660e+06,1.026491e+06
Contra Costa,4.930517e+05,4.933103e+05,4.948069e+05,4.986931e+05,5.027276e+05,5.048448e+05,4.924133e+05,4.906367e+05,4.878400e+05,4.845933e+05,...,8.055939e+05,8.121758e+05,8.167182e+05,8.203061e+05,8.233758e+05,8.294758e+05,8.373545e+05,8.444394e+05,8.498788e+05,8.523212e+05
Marin,9.519312e+05,9.558812e+05,9.541000e+05,9.451125e+05,9.381125e+05,9.360250e+05,9.321250e+05,9.291625e+05,9.266688e+05,9.215062e+05,...,1.428950e+06,1.430319e+06,1.431781e+06,1.435825e+06,1.440831e+06,1.451712e+06,1.470012e+06,1.481212e+06,1.483475e+06,1.484131e+06
Napa,4.507333e+05,4.523833e+05,4.507167e+05,4.430000e+05,4.354000e+05,4.338167e+05,4.330500e+05,4.320000e+05,4.286000e+05,4.272833e+05,...,7.781571e+05,7.912286e+05,7.991143e+05,7.973571e+05,7.909000e+05,7.927714e+05,8.044143e+05,8.165571e+05,8.273714e+05,8.363000e+05
San Francisco,1.149668e+06,1.153995e+06,1.150837e+06,1.143363e+06,1.142116e+06,1.144095e+06,1.141395e+06,1.136342e+06,1.131511e+06,1.128500e+06,...,2.054363e+06,2.067589e+06,2.088911e+06,2.099684e+06,2.104584e+06,2.120200e+06,2.136289e+06,2.147821e+06,2.170021e+06,2.192753e+06


In [4]:
# Assign each CountyName to its own dataset
alameda = df_grouped.loc['Alameda',]
contra_costa = df_grouped.loc['Contra Costa']
marin = df_grouped.loc['Marin',]
napa = df_grouped.loc['Napa',]
santa_clara = df_grouped.loc['Santa Clara',]
san_fran = df_grouped.loc['San Francisco',]
san_mateo = df_grouped.loc['San Mateo',]
sonoma = df_grouped.loc['Sonoma',]
solano = df_grouped.loc['Solano',]

In [5]:
# Split each CountyName to train and test sets
alameda_train, alameda_test = alameda[:int(len(alameda)*0.7)], alameda[int(len(alameda)*0.7):]
cc_train, cc_test = contra_costa[:int(len(contra_costa)*0.7)], contra_costa[int(len(contra_costa)*0.7):]
marin_train, marin_test = marin[:int(len(marin)*0.7)], marin[int(len(marin)*0.7):]
napa_train, napa_test = napa[:int(len(napa)*0.7)], napa[int(len(napa)*0.7):]
sc_train, sc_test = santa_clara[:int(len(santa_clara)*0.7)], santa_clara[int(len(santa_clara)*0.7):]
sf_train, sf_test = san_fran[:int(len(san_fran)*0.7)], san_fran[int(len(san_fran)*0.7):]
sm_train, sm_test = san_mateo[:int(len(san_mateo)*0.7)], san_mateo[int(len(san_mateo)*0.7):]
sonoma_train, sonoma_test = sonoma[:int(len(sonoma)*0.7)], sonoma[int(len(sonoma)*0.7):]
solano_train, solano_test = solano[:int(len(solano)*0.7)], solano[int(len(solano)*0.7):]